# Contact inhibition functions

This notebook goes through how contact inhibition functions can be used to quantify the number of neighbrs around a cell and decide to stop proliferation based on this value.

In [1]:
import numpy as np
import vedo

from neurorosettes.neurons import Neuron, NeuronFactory
from neurorosettes.clocks import ClocksFactory
from neurorosettes.subcellular import CellBody, ObjectFactory
from neurorosettes.simulation import Container, Timer
from neurorosettes import utilities
from neurorosettes import physics
from neurorosettes.grid import UniformGrid, OneLevelDensityCheck, TwoLevelsDensityCheck


def create_tissue() -> list:
    coordinates = [np.array([x + 8 * (i % 2), y, 0])
                   for x in range(-40, 41, 16)
                   for i, y in enumerate(range(-40, 41, 16))]
    return coordinates

## Creating objects and interactions

In [2]:
# Define the mechanical parameters of the objects
factory = ObjectFactory(
    cell_radius=8.0,
    cell_interaction_factor=1.25,
    neurite_radius=1.0,
    neurite_interaction_factor=2.0,
    neurite_spring_constant=10.0,
    neurite_default_length=15.0,
)

clocks = ClocksFactory(proliferation_rate=0.1, death_rate=0.1, differentiation_rate=0.1)

neuron_factory = NeuronFactory(4, factory, clocks)

# Define the mechanical parameters of the interactions
contact_function = physics.PotentialsFactory(sphere_sphere_adhesion=0.4,
                                             sphere_sphere_repulsion=10.0,
                                             sphere_sphere_smoothness=1,
                                             sphere_cylinder_adhesion=4.0,
                                             sphere_cylinder_repulsion=10.0,
                                             sphere_cylinder_smoothness=1,
                                             cylinder_cylinder_adhesion=4.0,
                                             cylinder_cylinder_repulsion=10.0,
                                             cylinder_cylinder_smoothness=1)

## One-level contact inhibition

The one-level contact inhibition function evaluates the number of neighbors in the current cell of the domain grid, as well as the cells surrounding it. This function evaluates if the number of cell bodies registered in these cells is larger than a user-defined value.

In [6]:
# Initialize simulation objects
container = Container(grid=UniformGrid(*[-70, 70, 20]),
                     simulation_2d=True,
                     neuron_factory=neuron_factory,
                     contact_factory=contact_function)

for position in create_tissue():
    # Populate environment with cells
    neuron = container.create_new_neuron(position)
    neuron.set_outgrowth_axis(utilities.get_random_unit_vector(two_dimensions=True))
    neuron.clocks.set_clocks(0.04, 0.00000001, 0.0000002)
    container.register_neuron(neuron)
    
neuron = container.neurons[29]
container.neurons[29].cell.sphere.c("red")

cell_neighbors = [neighbor
                  for neighbor in container.grid.get_close_cells(neuron.cell.position)]

for cell in cell_neighbors:
    if cell is not neuron.cell:
        cell.sphere.c("orange")

container.animator.plotter.show(resetcam=False, interactive=False)
container.animator.save_screenshot("../docs/img/one-level")

## Two-levels contact inhibition

The two-levels contact inhibition function performs the same evaluation as the one-level contact function and also checks the number of neighbors inside a user-defined radius.

In [5]:
container = Container(grid=UniformGrid(*[-70, 70, 20]),
                     simulation_2d=True,
                                          neuron_factory=neuron_factory,

                     contact_factory=contact_function)

for position in create_tissue():
    # Populate environment with cells
    neuron = container.create_new_neuron(position)
    neuron.set_outgrowth_axis(utilities.get_random_unit_vector(two_dimensions=True))
    neuron.clocks.set_clocks(0.04, 0.00000001, 0.0000002)
    container.register_neuron(neuron)
    
neuron = container.neurons[29]
container.neurons[29].cell.sphere.c("lightblue")

cell_neighbors = [neighbor
                  for neighbor in container.grid.get_close_cells(neuron.cell.position)]

for cell in cell_neighbors:
    if cell is not neuron.cell:
        cell.sphere.c("orange")
        
    if len(cell_neighbors) > 15:

        acceptable_radius = 20
        nearby = [neighbor for neighbor in cell_neighbors
                  if np.linalg.norm(neighbor.position - neuron.cell.position) <= acceptable_radius]

        for n in nearby:
            if n is not neuron.cell:
                n.sphere.c("yellow")

container.animator.plotter.show(resetcam=False, interactive=False)
container.animator.save_screenshot("../docs/img/two-level")